In [ ]:

# Set up the notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

In [ ]:

from FRVRS import (
    fu, nu, warnings, DataFrame, osp, listdir, Index, concat, read_excel, re, isna, to_datetime, display, NaT, read_csv, csv, to_numeric
)
import json

warnings.filterwarnings('ignore')


# JSON Stats Created for Metrics Evaluation Open World

Dave you should be ignoring all the files except the zip folder I sent you.
In the zip I sent you, there are 51 folders, (51 JSON, 51 CSV).
Zip file attached.
All the files are named appropriated in the folder/CSV/json UUID_ParticipantID.
Some of the internal Participants IDs might be off because the moderator forgot to enter a Participant ID or didn't enter the Participant ID correctly so we needed to figure out which participant it was.
Please only utilize the UUID and Participant ID that is on the file name to identify and ignore the internal Participant IDs.
Maybe that will help.

In [ ]:

# load data frames
data_frames_dict = nu.load_data_frames(metrics_evaluation_open_world_csv_stats_df='', first_responder_master_registry_file_stats_df='')
csv_stats_df = data_frames_dict['metrics_evaluation_open_world_csv_stats_df']
print(csv_stats_df.shape) # (171766, 111)

In [ ]:

# Get all the Open World JSON Stats into one data frame
json_stats_df = DataFrame([])
logs_path = osp.join(fu.data_folder, 'logs', 'Human_Sim_Metrics_Data_4-12-2024')
directories_list = listdir(logs_path)
for dir_name in directories_list:
    
    # Add the JSONs to the data frame
    folder_path = osp.join(logs_path, dir_name)
    
    # Iterate over the files in the current subdirectory
    for file_name in listdir(folder_path):
        
        # If the file is a JSON file
        if file_name.endswith('.json'):
            
            # Create a data frame from the flattened dictionary
            json_path = osp.join(folder_path, file_name)
            with open(json_path, 'r') as f: file_json = json.load(f)
            row_dict = {
                'json_file_subpath': folder_path,
                'json_file_name': file_name
            }
            flattened_json_dict = nu.get_flattened_dictionary(file_json, row_dict=row_dict, key_prefix='')
            
            # You've got to clean the Session IDs
            session_uuid, participant_id = dir_name.split('_')
            flattened_json_dict['session_uuid'] = session_uuid
            flattened_json_dict['participant_id'] = int(participant_id)
            df = DataFrame(flattened_json_dict, index=Index([0]))
            
            # Append the data frame for the current subdirectory to the main data frame and break the participant ID loop
            json_stats_df = concat([json_stats_df, df], axis='index')

json_stats_df = json_stats_df.reset_index(drop=True)
nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df)
nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df)
print(json_stats_df.participant_id.nunique()) # 22
print(json_stats_df.shape) # (43, 3525)


## Check for proper ingestion (duplicate file ingestion, et al)

In [ ]:

# Check you even ingested anything
assert len(json_stats_df.columns) > 4, "Nothing ingested"
assert json_stats_df.participant_id.nunique() == 26, f"Participant count should be 26, it's {json_stats_df.participant_id.nunique()} instead"
print(json_stats_df.shape) # (43, 3525)

In [ ]:

# load data frame
data_frames_dict = nu.load_data_frames(metrics_evaluation_open_world_json_stats_df='')
json_stats_df = data_frames_dict['metrics_evaluation_open_world_json_stats_df']

if 'logger_version' not in json_stats_df.columns:
    
    # Get the columns that consistently have only one value in them per session
    single_value_cols_set = set(csv_stats_df.columns)
    for session_uuid, session_df in csv_stats_df.groupby('session_uuid'):
        single_value_cols = set([col for col in session_df.columns if session_df[col].nunique() == 1])
        single_value_cols_set = single_value_cols_set.intersection(single_value_cols)
    print(single_value_cols_set)
    rows_list = []
    for session_uuid, session_df in csv_stats_df[list(single_value_cols_set)].dropna(axis='columns', how='all').groupby('session_uuid'):
        row_dict = {cn: session_df[cn].dropna().max() for cn in session_df.columns}
        rows_list.append(row_dict)
    single_value_cols_df = DataFrame(rows_list)
    print(single_value_cols_df.shape)
    
    on_columns = sorted(set(json_stats_df.columns).intersection(set(single_value_cols_df.columns)))
    print(on_columns)
    json_stats_df = json_stats_df.merge(
        single_value_cols_df, how='left', on=on_columns
    )
    print(json_stats_df.shape)
    print(single_value_cols_df.shape)
    
    # Save so you don't have to run it again
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df)
    
    print(json_stats_df.shape) # (43, 3538)

In [ ]:

# Get column and value descriptions
file_path = osp.join(fu.data_folder, 'xlsx', 'Metrics_Evaluation_Dataset_organization_for_BBAI.xlsx')
dataset_organization_df = read_excel(file_path)

# Fix the doubled up descriptions
mask_series = dataset_organization_df.Labels.map(lambda x: ';' in str(x))
for row_index, label in dataset_organization_df[mask_series].Labels.items():
    labels_list = re.split(' *; *', str(label), 0)
    dataset_organization_df.loc[row_index, 'Labels'] = labels_list[0]
    
    # Get a copy of the row
    new_row = dataset_organization_df.loc[row_index].copy()
    
    # Modify the desired column value
    new_row['Labels'] = labels_list[1]
    
    print("Append the new row to the DataFrame")
    dataset_organization_df = concat([dataset_organization_df, new_row], ignore_index=True)

# Get a copy of the row
mask_series = (dataset_organization_df.Variable == 'AD_Del_Omni')
new_row = dataset_organization_df.loc[mask_series].copy()

# Modify the desired column value
new_row['Variable'] = 'AD_Del_Omni_Text'

# Append the new row to the Data Frame
dataset_organization_df = concat([dataset_organization_df, new_row], ignore_index=True)

# Get the column value descriptions
mask_series = ~dataset_organization_df.Description.isnull()
df = dataset_organization_df[mask_series]
value_description_dict = df.set_index('Variable').Description.to_dict()
new_description_dict = value_description_dict.copy()
for k, v in value_description_dict.items():
    new_description_dict[k] = v
    if (not k.endswith('_Text')):
        new_key_name = f'{k}_Text'
        new_description_dict[new_key_name] = new_description_dict.get(new_key_name, v)
value_description_dict = new_description_dict.copy()

# Create the value description function
numeric_categories_mask_series = dataset_organization_df.Labels.map(lambda x: '=' in str(x))
value_descriptions_columns = dataset_organization_df[numeric_categories_mask_series].Variable.unique().tolist()
def get_value_description(column_name, column_value):
    value_description = ''
    if not isna(column_value):
        mask_series = (dataset_organization_df.Variable == column_name) & ~dataset_organization_df.Labels.isnull()
        if mask_series.any():
            df = dataset_organization_df[mask_series]
            mask_series = df.Labels.map(lambda label: re.split(' *= *', str(label), 0)[0] == str(int(float(column_value))))
            if mask_series.any():
                label = df[mask_series].Labels.squeeze()
                value_description = re.split(' *= *', str(label), 0)[1]
    
    return value_description

In [ ]:

# load data frame
data_frames_dict = nu.load_data_frames(metrics_evaluation_open_world_json_stats_df='')
json_stats_df = data_frames_dict['metrics_evaluation_open_world_json_stats_df']

if 'MedExp' not in json_stats_df.columns:
    file_path = osp.join(fu.data_folder, 'xlsx', 'participant_data_0420.xlsx')
    participant_data_df = read_excel(file_path).rename(columns={'ParticipantID': 'participant_id', 'Date': 'participation_date'})
    participant_data_df.participation_date = to_datetime(participant_data_df.participation_date, format='%m/%d/%Y')
    
    # See if you can merge on participant_id
    on_columns = sorted(set(json_stats_df.columns).intersection(set(participant_data_df.columns)))
    print(on_columns)
    if on_columns:
        print(json_stats_df.shape)
        json_stats_df = json_stats_df.merge(
            participant_data_df, how='left', on=on_columns
        )
        print(participant_data_df.shape)
        # print(sorted(set(participant_data_df.groupby('participant_id').mean().columns).intersection(set(
        #     dataset_organization_df[~numeric_categories_mask_series].Variable
        # ))))
        print(json_stats_df.shape)
    
    # Check if the various partipant id columns are inconsistent
    columns_list = [cn for cn in json_stats_df.columns if cn.lower().startswith('participant') and cn.lower().endswith('id')]
    df = json_stats_df[columns_list]
    for cn in columns_list:
        df[cn] = df[cn].map(lambda x: str(x).strip())
    mask_series = (df[columns_list[0]] != df[columns_list[1]])
    if mask_series.any(): print("The various partipant id columns are inconsistent")
    
    # Save so you don't have to run it again
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df)

In [ ]:

# Remove any duplicate session IDs
mask_series = json_stats_df.duplicated(subset='session_uuid')
if mask_series.any():
    display(json_stats_df[mask_series].dropna(axis='columns', how='all')); raise
    json_stats_df = json_stats_df[~mask_series]
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df)
    print(json_stats_df.shape) # (60, 8)

In [ ]:

# Fix the null file dates
if 'session_file_date' not in json_stats_df.columns:
    json_stats_df['session_file_date'] = NaT
mask_series = json_stats_df.session_file_date.isnull()
if mask_series.any():
    print(f'I have {mask_series.sum()} sessions in my JSON Stats data frame without file dates.')
    for session_uuid, idx_df in json_stats_df[mask_series].groupby('session_uuid'):
        
        # Get the whole session history
        mask_series = (csv_stats_df.session_uuid == session_uuid)
        if mask_series.any():
            session_df = csv_stats_df[mask_series]
            
            mask_series = ~session_df.event_time.isnull()
            assert mask_series.any(), f"Session ID {session_uuid} doesn't have an event times in csv_stats_df"
            session_file_date = session_df[mask_series].event_time.min().date()
            json_stats_df.loc[idx_df.index, 'session_file_date'] = session_file_date
        else: print(f"You're missing a Session ID {session_uuid} in csv_stats_df")
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df)
    mask_series = json_stats_df.session_file_date.isnull()
    if mask_series.any():
        print(f'I still have {mask_series.sum()} sessioms in my JSON Stats data frame without file dates.')
        display(json_stats_df[mask_series].dropna(axis='columns', how='all').T)

In [ ]:

# Add file start time
if ('file_start_time' not in json_stats_df.columns) or ('file_stop_time' not in json_stats_df.columns):
    if ('json_file_name' in json_stats_df.columns) and ('json_file_subpath' in json_stats_df.columns):
        for (file_name, folder_path), idx_df in json_stats_df.groupby(['json_file_name', 'json_file_subpath']):
            
            # Construct the full path to the file
            file_path = osp.join(folder_path, file_name)
            
            # Attempt to read CSV file using pandas; if unsuccessful, try using a reader
            try: file_df = read_csv(file_path, header=None, index_col=False)
            except:
                rows_list = []
                with open(file_path, 'r') as f:
                    reader = csv.reader(f, delimiter=',', quotechar='"')
                    for values_list in reader:
                        if (values_list[-1] == ''): values_list.pop(-1)
                        rows_list.append({i: v for i, v in enumerate(values_list)})
                file_df = DataFrame(rows_list)
            
            ts_series = to_datetime(file_df[2], infer_datetime_format=True, errors='coerce')
            if ts_series.dropna().shape[0]:
                json_stats_df.loc[idx_df.index, 'file_start_time'] = ts_series.dropna().min().to_pydatetime()
                json_stats_df.loc[idx_df.index, 'file_stop_time'] = ts_series.dropna().max().to_pydatetime()
        print(json_stats_df.shape) # (43, 3538)
        nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df)
        nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df)

In [ ]:

# Get a sample with a clear count of responders
new_column_name = 'is_a_one_triage_file'
# if (new_column_name in json_stats_df.columns):
    # json_stats_df = json_stats_df.drop(columns=new_column_name)
if (new_column_name not in json_stats_df.columns):
    json_stats_df[new_column_name] = False
    data_frames_dict = nu.load_data_frames(metrics_evaluation_open_world_scene_stats_df='')
    scene_stats_df = data_frames_dict['metrics_evaluation_open_world_scene_stats_df']
    if scene_stats_df is not None:
        
        # Assume a 1:1 correspondence between file name and UUID from the logs data frame build
        for session_uuid, session_df in json_stats_df.groupby('session_uuid'):
            assert session_df.shape[0] == 1, "You've got duplicate session UUIDs"
            
            # Filter in the triage files in this UUID
            mask_series = (scene_stats_df.session_uuid == session_uuid) & (scene_stats_df.scene_type == 'Triage')
            
            # Get whether the file has only one triage run
            triage_scene_count = len(scene_stats_df[mask_series].groupby('scene_id').groups)
            is_a_one_triage_file = bool(triage_scene_count == 1)
            
            json_stats_df.loc[session_df.index, new_column_name] = is_a_one_triage_file
        
        # Store the results and show the new data frame shape
        nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df)
        nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df)
        print(json_stats_df.shape) # (43, 3539)

display(json_stats_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

In [ ]:

new_column_name = 'is_in_registry'
if (new_column_name not in json_stats_df.columns):
    json_stats_df[new_column_name] = False
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df)
    print(json_stats_df.shape) # (43, 3540)

display(json_stats_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

In [ ]:

# Convert the participant ID columns to numeric
columns_list = [cn for cn in json_stats_df.columns if 'partici' in cn.lower()]
for cn in columns_list: json_stats_df[cn] = to_numeric(json_stats_df[cn], errors='coerce')
print(json_stats_df.shape) # (43, 3570)
nu.store_objects(metrics_evaluation_open_world_json_stats_df=json_stats_df)
nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=json_stats_df)